# Analisi sulla somministrazione dei vaccini

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

## Parametri

In [2]:
data_file='https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/somministrazioni-vaccini-summary-latest.csv'

## Lettura dati e preprocessing

In [3]:
vaccinazioni_data=pd.read_csv(data_file)
print(vaccinazioni_data.columns)

# riordino in base alla data (che creo e faccio indice)
vaccinazioni_data['data'] = pd.to_datetime(vaccinazioni_data['data_somministrazione'])
vaccinazioni_data.index=vaccinazioni_data['data']
vaccinazioni_data=vaccinazioni_data.sort_index()


# creo una colonna col numero di giorni dalla prima consegna
zero_day=vaccinazioni_data['data'][0]
day_counter=[(dd-zero_day).days for dd in vaccinazioni_data['data']]
vaccinazioni_data['day_counter']=day_counter

giorni=pd.unique(vaccinazioni_data['data_somministrazione'])
day_counter=list(range(len(giorni)))
print('Giorni osservati: %d'%len(giorni))
print('Da: %s'%giorni[0][0:10])
print(' A: %s'%giorni[-1][0:10])
regioni=pd.unique(vaccinazioni_data['area'])


#data[data['denominazione_regione']=='Lombardia'].head(5)
vaccinazioni_data[vaccinazioni_data['area']=='TOS'].tail(5)


Index(['data_somministrazione', 'area', 'totale', 'sesso_maschile',
       'sesso_femminile', 'categoria_operatori_sanitari_sociosanitari',
       'categoria_personale_non_sanitario', 'categoria_ospiti_rsa',
       'categoria_over80', 'categoria_forze_armate',
       'categoria_personale_scolastico', 'prima_dose', 'seconda_dose',
       'codice_NUTS1', 'codice_NUTS2', 'codice_regione_ISTAT', 'nome_area'],
      dtype='object')
Giorni osservati: 86
Da: 2020-12-27
 A: 2021-03-22


,data_somministrazione,area,totale,sesso_maschile,sesso_femminile,categoria_operatori_sanitari_sociosanitari,categoria_personale_non_sanitario,categoria_ospiti_rsa,categoria_over80,categoria_forze_armate,categoria_personale_scolastico,prima_dose,seconda_dose,codice_NUTS1,codice_NUTS2,codice_regione_ISTAT,nome_area,data,day_counter
data,,,,,,,,,,,,,,,,,,,
2021-03-18,2021-03-18,TOS,9536,3806,5730,427,1416,256,7427,7,3,5682,3854,ITI,ITI1,9,Toscana,2021-03-18,81
2021-03-19,2021-03-19,TOS,9740,4296,5444,618,4422,241,4411,13,35,7058,2682,ITI,ITI1,9,Toscana,2021-03-19,82
2021-03-20,2021-03-20,TOS,6325,2781,3544,879,5290,80,28,14,34,5229,1096,ITI,ITI1,9,Toscana,2021-03-20,83
2021-03-21,2021-03-21,TOS,3221,1571,1650,657,2466,73,7,6,12,1794,1427,ITI,ITI1,9,Toscana,2021-03-21,84
2021-03-22,2021-03-22,TOS,956,458,498,108,730,66,2,11,39,782,174,ITI,ITI1,9,Toscana,2021-03-22,85


### Creazione della struttura dati


**MODIFICARE**

- Si crea un dizionario per ciascuno degli indicatori selezionati
- a sua volta ciascun elemento del primo dizionario è un dizionario con le regioni come chiave
    - ogni elemento indicatore contiene i valori dell'indicatore per la regione

**Esempio:**
```    
trend[indicatore][regione]
```

In [4]:
# indicatori osservati
indicatori=['data','day_counter','totale', 'sesso_maschile',
       'sesso_femminile', 'categoria_operatori_sanitari_sociosanitari',
       'categoria_personale_non_sanitario', 'categoria_ospiti_rsa',
       'categoria_over80', 'categoria_forze_armate',
       'categoria_personale_scolastico', 'prima_dose', 'seconda_dose']

# qui immetto tutti i trend per ciascuna regione
vaccinazioni={}
for regione in regioni:
    vaccinazioni[regione]={}
    for ind in indicatori:    
        vaccinazioni[regione][ind]=np.array(vaccinazioni_data[vaccinazioni_data['area']==regione][ind])
      

Aggrego nell'indicatore **ITALIA**

In [5]:
a=np.where(vaccinazioni['TOS']['day_counter']==1)
a[0][0]

IndexError: index 0 is out of bounds for axis 0 with size 0

## Grafici